In [1]:
library(SnapATAC)
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(parallel)
library(foreach)
Sys.setenv(OPENBLAS_NUM_THREADS=1)

Loading required package: Matrix

Loading required package: rhdf5

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects ar

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/pairs.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/matrix.atac.5_super_groups.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/5.merge_peaks_from_5_super_groups.240524/matrix.rna.5_super_groups.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SnapATAC/BMMC5.merge.250212/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac_count = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac_count)
rm(matrix.atac_count)

Import RNA matrix

In [6]:
matrix.rna_count = readRDS(path.matrix.rna_count)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna_count = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked

In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]

Run SnapATAC prediction

In [10]:
PredictionSnapATAC = function(pairs.E2G,
                              data.RNA,
                              data.ATAC,
                              n.core = 16){
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (index.tmp = 1:length(pairs.E2G),
             .combine = 'c',
             .packages = c("Matrix",
                           "GenomicRanges")) %dopar% {

                             pairs.E2G.tmp = pairs.E2G[index.tmp]
                             
                             model.tmp = summary(stats::glm(data.ATAC[pairs.E2G.tmp$PeakName,] ~ data.RNA[pairs.E2G.tmp$TargetGene,], 
                                                            family = binomial(link = "logit")))[["coefficients"]]
                             mcols(pairs.E2G.tmp)[,c("beta","sd","z","p")] = 
                               t(data.frame(model.tmp[2,]))
                             
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

Warning message in dir.create(paste(dir.output, "chr", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SnapATAC/BMMC5.merge.250212//chr' already exists”


In [12]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter)))
chr.run = chr.run[!chr.run %in% chr.done]

In [13]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter[seqnames(pairs.E2G.filter) == chr.tmp]
  mcols(pairs.E2G.chr.res)[,c("beta","sd","z","p")] = NA
  pairs.E2G.chr.res = PredictionSnapATAC(pairs.E2G.chr.res,
                                         matrix.rna.rename[rownames(matrix.rna.rename) %in% pairs.E2G.chr.res$TargetGene,],
                                         matrix.atac[rownames(matrix.atac) %in% pairs.E2G.chr.res$PeakName,],
                                         n.cores)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"
[1] "chr10"
[1] "chr11"
[1] "chr12"
[1] "chr13"
[1] "chr14"
[1] "chr15"
[1] "chr16"
[1] "chr17"
[1] "chr18"
[1] "chr19"
[1] "chr2"
[1] "chr20"
[1] "chr21"
[1] "chr22"
[1] "chr3"
[1] "chr4"
[1] "chr5"
[1] "chr6"
[1] "chr7"
[1] "chr8"
[1] "chr9"
[1] "chrX"


Merge results

In [14]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [15]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 5660858 ranges and 7 metadata columns:
       seqnames              ranges strand |  TargetGene               PeakName
          <Rle>           <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 100015450-100016026      * |         AGL chr1-100015450-10001..
  chr1     chr1 100015450-100016026      * |      CDC14A chr1-100015450-10001..
  chr1     chr1 100015450-100016026      * |         DBT chr1-100015450-10001..
  chr1     chr1 100015450-100016026      * |       EXTL2 chr1-100015450-10001..
  chr1     chr1 100015450-100016026      * |     MFSD14A chr1-100015450-10001..
   ...      ...                 ...    ... .         ...                    ...
  chrX     chrX     9995615-9996434      * |        WWC3   chrX-9995615-9996434
  chrX     chrX     9997010-9998101      * |       CLCN4   chrX-9997010-9998101
  chrX     chrX     9997010-9998101      * |     SHROOM2   chrX-9997010-9998101
  chrX     chrX     9997010-9998101      * |       TBL1X   ch

In [16]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta",
                         "sd",
                         "z",
                         "p")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,sd,z,p
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100015450,100016026,AGL,K562,0.186202797,0.143923884,1.29375884,1.957487e-01
chr1,100015450,100016026,CDC14A,K562,0.133354374,0.068663708,1.94213768,5.212043e-02
chr1,100015450,100016026,DBT,K562,-0.411851772,0.288115031,-1.42946992,1.528692e-01
chr1,100015450,100016026,EXTL2,K562,0.265306585,0.306112058,0.86669760,3.861077e-01
chr1,100015450,100016026,MFSD14A,K562,0.163396983,0.489481785,0.33381627,7.385182e-01
chr1,100015450,100016026,RTCA,K562,0.090986595,0.136889000,0.66467426,5.062589e-01
chr1,100015450,100016026,RTCA-AS1,K562,0.014138722,0.413738974,0.03417305,9.727392e-01
chr1,100015450,100016026,SASS6,K562,0.015875834,0.183221132,0.08664849,9.309509e-01
chr1,100015450,100016026,SLC30A7,K562,0.101410986,0.100720610,1.00685437,3.140048e-01


In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2    dplyr_1.1.4          foreach_1.5.2       
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges